Op verzoek van het RIVM worden enkele van de grotere bestanden voortaal 'opgesplitst' aangeleverd, d.w.z. in een eenmalig 'statisch' bestand met data van voor 4-10-2021 en in een dagelijks bestand dat de data bevat vanaf 4-10-2021. Deze opsplitsing is gewenst voor initieel 9 bestanden en op termijn zo'n 30 bestanden. 

Hier wordt dit allereerst gedaan voor een eerste groep van 9 bestanden. Eerst wordt er een lege tabel aangemaakt in het VWSSTATIC schema, zodat daarin eenmalig de statische gegevens kunnen worden ingelezen. Daarna wordt de stored procedure aangepast die de gegevens van VWSSTAGE naar VWSINTER kopieert. 

Bij de aanpassing van de stored procedure is van belang om goed om te gaan met de kolom Date\_last\_inserted.  In de VWSINTER tabellen wordt deze standaard automatisch gevuld met een tijdstempel via GetDate(). Dat gaat goed als er vanuit één enkele bron wordt ingelezen (die krijgt dan voor de hele bron dezelfde tijdstempel). Hier wordt echter ingelezen vanuit twee bronnen, in twee verschillende insert-operaties met elk een eigen tijdstempel. Zonder aanpassingen krijgen de twee bronnen twee verschillende tijdstempels. In de verdere verwerking wordt steeds gefilterd op de Max() van de Date\_Last\_Inserted en dat zou betekenen dat alleen de tweede insert wordt doorgegeven. Om dit te voorkomen is de automatische vulling van de kolom Date\_Last\_Inserted uitgezet, wordt voor de eerste insert de tijdstempel in een variabele gezet en wordt voor beide inserts deze variabele meegegeven als de bijbehorende tijdstempel.

De tabellen die zo worden aangepast zijn:

1. RIVM\_COVID\_19\_NUMBER\_MUNICIPALITY
    
2. DISABILITY
    
3. ELDERLY\_AT\_HOME
    
4. GGD\_TESTED\_PEOPLE
    
5. NURSING\_HOMES\_COMBINED
    
6. RIVM\_REPRODUCTION\_NUMBER
    
7. RIVM\_IC\_OPNAME
    
8. NICE\_HOSPITAL
    
9. RIVM\_HOSPITAL\_IC\_ADMISSIONS\_OVERTIME\_BYAGEGROUP

In [ ]:


IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_COVID_19_NUMBER_MUNICIPALITY'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_COVID_19_NUMBER_MUNICIPALITY](
		[ID] INT IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_PUBLICATION] [varchar](100) NULL,
		[MUNICIPALITY_CODE] [varchar](100) NULL,
		[MUNICIPALITY_NAME] [varchar](100) NULL,
		[PROVINCE] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
		[ROAZ_REGION] [varchar](100) NULL,
		[TOTAL_REPORTED] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[DECEASED] [varchar](100) NULL,
		[VERSION] [varchar](50) NULL,
		[DATE_LAST_INSERTED] DateTime Default GetDate()
	) ON [PRIMARY]
END
GO

In [ ]:
-- Verwijderen default waarde Date_Last_Inserted in VWSINTER, omdat er nu data komt vanuit twee bronnen die potentieel verschillende Date_last_Inserted kunnen krijgen.
ALTER TABLE [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_COVID_19_MUNICIPALITY_INTER]
 AS
 BEGIN

    -- Zet de Date_Last_Inserted in een variabele
    DECLARE @DateTimeNow AS DateTime = GETDATE()

    -- Voeg de regels toe vanuit VWSSTAGE
    INSERT INTO [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY]
     (
         [DATE_OF_REPORT],
         [DATE_OF_PUBLICATION],
         [MUNICIPALITY_CODE],
         [MUNICIPALITY_NAME],
         [SECURITY_REGION_CODE],
         [SECURITY_REGION_NAME],
         [PROVINCE],
         [MUNICIPAL_HEALTH_SERVICE],
         [ROAZ_REGION],
         [TOTAL_REPORTED],
         [HOSPITAL_ADMISSION],
         [DECEASED],
         [Date_Last_Inserted]
     )
     SELECT 
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT],
         CAST(A.[DATE_OF_PUBLICATION] AS [DATETIME]) AS [DATE_OF_REPORT],
         ISNULL(B.[GM_CODE_NEW], TRIM(A.[MUNICIPALITY_CODE])),
         ISNULL(B.[GM_NAME_NEW], TRIM(A.[MUNICIPALITY_NAME])),
         ISNULL(B.[VR_CODE], TRIM(A.[SECURITY_REGION_CODE])),
         ISNULL(B.[VR_NAME], TRIM(A.[SECURITY_REGION_NAME])),
         ISNULL(B.[PROVINCE_CODE], TRIM(A.[PROVINCE])),
         ISNULL(B.[GGD_NAME], TRIM(A.[MUNICIPAL_HEALTH_SERVICE])),
         TRIM(A.[ROAZ_REGION]) AS [ROAZ_REGION],
         CAST(SUM(CAST(A.[TOTAL_REPORTED] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [TOTAL_REPORTED],
         CAST(SUM(CAST(A.[HOSPITAL_ADMISSION] AS [FLOAT])* ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITAL_ADMISSION],
         CAST(SUM(CAST(A.[DECEASED] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [DECEASED],
         @DateTimeNow
     FROM  [VWSSTAGE].[RIVM_COVID_19_NUMBER_MUNICIPALITY] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_COVID_19_NUMBER_MUNICIPALITY])
     GROUP BY ISNULL(B.[GM_CODE_NEW], TRIM(A.[MUNICIPALITY_CODE])),
         ISNULL(B.[GM_NAME_NEW], TRIM(A.[MUNICIPALITY_NAME])),
         ISNULL(B.[VR_CODE], TRIM(A.[SECURITY_REGION_CODE])),
         ISNULL(B.[VR_NAME], TRIM(A.[SECURITY_REGION_NAME])),
         ISNULL(B.[PROVINCE_CODE], TRIM(A.[PROVINCE])),
         ISNULL(B.[GGD_NAME], TRIM(A.[MUNICIPAL_HEALTH_SERVICE])),
         TRIM(A.[ROAZ_REGION]),
         A.[DATE_OF_REPORT],
         A.[DATE_OF_PUBLICATION]
 
  -- Voeg nu de regels toe uit VWSSTATIC
  INSERT INTO [VWSINTER].[RIVM_COVID_19_NUMBER_MUNICIPALITY]
     (
         [DATE_OF_REPORT],
         [DATE_OF_PUBLICATION],
         [MUNICIPALITY_CODE],
         [MUNICIPALITY_NAME],
         [SECURITY_REGION_CODE],
         [SECURITY_REGION_NAME],
         [PROVINCE],
         [MUNICIPAL_HEALTH_SERVICE],
         [ROAZ_REGION],
         [TOTAL_REPORTED],
         [HOSPITAL_ADMISSION],
         [DECEASED],
         [Date_Last_Inserted]
     )
     SELECT 
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT],
         CAST(A.[DATE_OF_PUBLICATION] AS [DATETIME]) AS [DATE_OF_REPORT],
         ISNULL(B.[GM_CODE_NEW], TRIM(A.[MUNICIPALITY_CODE])),
         ISNULL(B.[GM_NAME_NEW], TRIM(A.[MUNICIPALITY_NAME])),
         ISNULL(B.[VR_CODE], TRIM(A.[SECURITY_REGION_CODE])),
         ISNULL(B.[VR_NAME], TRIM(A.[SECURITY_REGION_NAME])),
         ISNULL(B.[PROVINCE_CODE], TRIM(A.[PROVINCE])),
         ISNULL(B.[GGD_NAME], TRIM(A.[MUNICIPAL_HEALTH_SERVICE])),
         TRIM(A.[ROAZ_REGION]) AS [ROAZ_REGION],
         CAST(SUM(CAST(A.[TOTAL_REPORTED] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [TOTAL_REPORTED],
         CAST(SUM(CAST(A.[HOSPITAL_ADMISSION] AS [FLOAT])* ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITAL_ADMISSION],
         CAST(SUM(CAST(A.[DECEASED] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [DECEASED],
         @DateTimeNow
     FROM  [VWSSTATIC].[RIVM_COVID_19_NUMBER_MUNICIPALITY] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_COVID_19_NUMBER_MUNICIPALITY])
     GROUP BY ISNULL(B.[GM_CODE_NEW], TRIM(A.[MUNICIPALITY_CODE])),
         ISNULL(B.[GM_NAME_NEW], TRIM(A.[MUNICIPALITY_NAME])),
         ISNULL(B.[VR_CODE], TRIM(A.[SECURITY_REGION_CODE])),
         ISNULL(B.[VR_NAME], TRIM(A.[SECURITY_REGION_NAME])),
         ISNULL(B.[PROVINCE_CODE], TRIM(A.[PROVINCE])),
         ISNULL(B.[GGD_NAME], TRIM(A.[MUNICIPAL_HEALTH_SERVICE])),
         TRIM(A.[ROAZ_REGION]),
         A.[DATE_OF_REPORT],
         A.[DATE_OF_PUBLICATION]

 END;
GO


In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'DISABILITY'))
BEGIN
	CREATE TABLE [VWSSTATIC].[DISABILITY](
		[ID] INT IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[Date_Last_Inserted] DateTime Default GetDate()
	) 
END

In [ ]:
ALTER TABLE [VWSINTER].[DISABILITY] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_Disability_Inter]
AS
BEGIN

   -- Zet de Date_Last_Inserted in een variabele
    DECLARE @DateTimeNow AS DateTime = GETDATE()

    INSERT INTO VWSINTER.DISABILITY (
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        Date_Last_Inserted
        )
    SELECT
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        @DateTimeNow
    FROM
        VWSSTAGE.DISABILITY
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.DISABILITY)

    INSERT INTO VWSINTER.DISABILITY (
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        Date_Last_Inserted
        )
    SELECT
        DATE_OF_REPORT,
        DATE_OF_STATISTIC_REPORTED,
        SECURITY_REGION_CODE,
        TOTAL_CASES_REPORTED,
        TOTAL_DECEASED_REPORTED,
        TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,
        TOTAL_INFECTED_LOCATIONS_REPORTED,
        @DateTimeNow
    FROM
        VWSSTATIC.DISABILITY
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTATIC.DISABILITY)


END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'ELDERLY_AT_HOME'))
BEGIN

	CREATE TABLE [VWSSTATIC].[ELDERLY_AT_HOME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSINTER].[ELDERLY_AT_HOME] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE OR ALTER PROCEDURE [dbo].[SP_Elderly_At_Home_Inter]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.ELDERLY_AT_HOME (
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    Date_Last_Inserted
    )
SELECT
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    @DateTimeNow
FROM VWSSTAGE.ELDERLY_AT_HOME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.ELDERLY_AT_HOME)


INSERT INTO VWSINTER.ELDERLY_AT_HOME (
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    Date_Last_Inserted
    )
SELECT
    DATE_OF_REPORT,
    DATE_OF_STATISTIC_REPORTED,
    SECURITY_REGION_CODE,
    TOTAL_CASES_REPORTED,
    TOTAL_DECEASED_REPORTED,
    @DateTimeNow
FROM VWSSTATIC.ELDERLY_AT_HOME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.ELDERLY_AT_HOME)


END;
GO


In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'GGD_TESTED_PEOPLE'))
BEGIN

	CREATE TABLE [VWSSTATIC].[GGD_TESTED_PEOPLE](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TESTED_WITH_RESULT] [varchar](100) NULL,
		[TESTED_POSITIVE] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate() 
	) ON [PRIMARY]
END


In [ ]:
ALTER TABLE [VWSINTER].[GGD_TESTED_PEOPLE] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_GGD_TESTED_PEOPLE_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.GGD_TESTED_PEOPLE
    (
       [VERSION],
       DATE_OF_REPORT,
       DATE_OF_STATISTICS,
       SECURITY_REGION_CODE,
       SECURITY_REGION_NAME,
       TESTED_WITH_RESULT,
       TESTED_POSITIVE,
       Date_Last_Inserted
    )
    SELECT
        [VERSION],
        DATE_OF_REPORT,
        DATE_OF_STATISTICS,
        SECURITY_REGION_CODE,
        SECURITY_REGION_NAME,
        TESTED_WITH_RESULT,
        TESTED_POSITIVE,
        @DateTimeNow as Date_Last_Inserted
    FROM
       VWSSTAGE.GGD_TESTED_PEOPLE
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.GGD_TESTED_PEOPLE)

INSERT INTO VWSINTER.GGD_TESTED_PEOPLE
    (
       [VERSION],
       DATE_OF_REPORT,
       DATE_OF_STATISTICS,
       SECURITY_REGION_CODE,
       SECURITY_REGION_NAME,
       TESTED_WITH_RESULT,
       TESTED_POSITIVE,
       Date_Last_Inserted
    )
    SELECT
        [VERSION],
        DATE_OF_REPORT,
        DATE_OF_STATISTICS,
        SECURITY_REGION_CODE,
        SECURITY_REGION_NAME,
        TESTED_WITH_RESULT,
        TESTED_POSITIVE,
        @DateTimeNow as Date_Last_Inserted
    FROM
       VWSSTATIC.GGD_TESTED_PEOPLE
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.GGD_TESTED_PEOPLE)    

END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_NURSING_HOMES_COMBINED'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_NURSING_HOMES_COMBINED](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTIC_REPORTED] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[TOTAL_CASES_REPORTED] [varchar](100) NULL,
		[TOTAL_DECEASED_REPORTED] [varchar](100) NULL,
		[TOTAL_NEW_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[TOTAL_INFECTED_LOCATIONS_REPORTED] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSSTATIC].[RIVM_NURSING_HOMES_COMBINED] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE OR ALTER  PROCEDURE [dbo].[SP_RIVM_NURSING_HOMES_COMBINED_INTER]
AS
BEGIN
-- Main select and insert into statement. Filtered values on max datelastinserted.
-- Move nursery intake data from staging to intermediate table. 

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_NURSING_HOMES_COMBINED
    (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTIC_REPORTED],
        [SECURITY_REGION_CODE],
        [SECURITY_REGION_NAME],
        [TOTAL_CASES_REPORTED],
        [TOTAL_DECEASED_REPORTED],
        [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        [TOTAL_INFECTED_LOCATIONS_REPORTED],
        [DATE_LAST_INSERTED]
    )
    SELECT
        -- Provided datetime format in source file is yyyy-mm-dd
        CONVERT(DATETIME, [DATE_OF_REPORT], 102),
        CONVERT(DATETIME, [DATE_OF_STATISTIC_REPORTED], 102),
        TRIM([SECURITY_REGION_CODE]),
        [SECURITY_REGION_NAME],
        ISNULL(TOTAL_CASES_REPORTED,0) AS [TOTAL_CASES_REPORTED],
        ISNULL(TOTAL_DECEASED_REPORTED,0) AS [TOTAL_DECEASED_REPORTED],
        ISNULL(TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        ISNULL(TOTAL_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_INFECTED_LOCATIONS_REPORTED],
        @DateTimeNow as [DATE_LAST_INSERTED]
    FROM VWSSTAGE.RIVM_NURSING_HOMES_COMBINED
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.RIVM_NURSING_HOMES_COMBINED)


INSERT INTO VWSINTER.RIVM_NURSING_HOMES_COMBINED
    (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTIC_REPORTED],
        [SECURITY_REGION_CODE],
        [SECURITY_REGION_NAME],
        [TOTAL_CASES_REPORTED],
        [TOTAL_DECEASED_REPORTED],
        [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        [TOTAL_INFECTED_LOCATIONS_REPORTED],
        [DATE_LAST_INSERTED]
    )
    SELECT
        -- Provided datetime format in source file is yyyy-mm-dd
        CONVERT(DATETIME, [DATE_OF_REPORT], 102),
        CONVERT(DATETIME, [DATE_OF_STATISTIC_REPORTED], 102),
        TRIM([SECURITY_REGION_CODE]),
        [SECURITY_REGION_NAME],
        ISNULL(TOTAL_CASES_REPORTED,0) AS [TOTAL_CASES_REPORTED],
        ISNULL(TOTAL_DECEASED_REPORTED,0) AS [TOTAL_DECEASED_REPORTED],
        ISNULL(TOTAL_NEW_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_NEW_INFECTED_LOCATIONS_REPORTED],
        ISNULL(TOTAL_INFECTED_LOCATIONS_REPORTED,0) AS [TOTAL_INFECTED_LOCATIONS_REPORTED],
        @DateTimeNow as [DATE_LAST_INSERTED]
    FROM VWSSTATIC.RIVM_NURSING_HOMES_COMBINED
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.RIVM_NURSING_HOMES_COMBINED)



END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_REPRODUCTION_NUMBER'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_REPRODUCTION_NUMBER](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[DATE] [varchar](100) NULL,
		[RT_LOW] [varchar](100) NULL,
		[RT_AVG] [varchar](100) NULL,
		[RT_UP] [varchar](100) NULL,
		[POPULATION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate(),
		[VERSION] [varchar](20) NULL
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSSTAGE].[RIVM_REPRODUCTION_NUMBER] DROP CONSTRAINT [DATE_LAST_INSERTED]
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- Move reproduction data from staging to intermediate table.
CREATE   PROCEDURE [dbo].[SP_RIVM_REPRODUCTION_NUMBER_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_REPRODUCTION_NUMBER
        ([DATE],
        RT_LOW,
        RT_AVG,
        RT_UP,
        DATE_LAST_INSERTED )
SELECT 
        [DATE],
        (CASE WHEN RT_LOW = '' THEN NULL ELSE RT_LOW END) AS RT_LOW,
        (CASE WHEN RT_AVG = '' THEN NULL ELSE RT_AVG END) AS RT_AVG,
        (CASE WHEN RT_UP = '' THEN NULL ELSE RT_UP END) AS RT_UP,
        @DateTimeNow as Date_Last_Inserted
    FROM 
       VWSSTAGE.RIVM_REPRODUCTION_NUMBER
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.RIVM_REPRODUCTION_NUMBER)


INSERT INTO VWSINTER.RIVM_REPRODUCTION_NUMBER
        ([DATE],
        RT_LOW,
        RT_AVG,
        RT_UP,
        DATE_LAST_INSERTED )
SELECT 
        [DATE],
        (CASE WHEN RT_LOW = '' THEN NULL ELSE RT_LOW END) AS RT_LOW,
        (CASE WHEN RT_AVG = '' THEN NULL ELSE RT_AVG END) AS RT_AVG,
        (CASE WHEN RT_UP = '' THEN NULL ELSE RT_UP END) AS RT_UP,
        @DateTimeNow as Date_Last_Inserted
    FROM 
       VWSSTATIC.RIVM_REPRODUCTION_NUMBER
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTATIC.RIVM_REPRODUCTION_NUMBER)

END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_IC_OPNAME'))
BEGIN


	CREATE TABLE [VWSSTATIC].[RIVM_IC_OPNAME](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSINTER].[RIVM_IC_OPNAME] DROP CONSTRAINT [DATE_LAST_INSERTED]
GO

In [ ]:
CREATE   PROCEDURE [dbo].[SP_RIVM_IC_OPNAME_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO VWSINTER.RIVM_IC_OPNAME(
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    Date_Last_Inserted 
    )
SELECT
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    @DateTimeNow as Date_Last_Inserted 
FROM 
        VWSSTAGE.RIVM_IC_OPNAME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.RIVM_IC_OPNAME)


INSERT INTO VWSINTER.RIVM_IC_OPNAME(
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    Date_Last_Inserted 
    )
SELECT
    [VERSION],
	DATE_OF_REPORT,
    DATE_OF_STATISTICS,
    IC_ADMISSION_NOTIFICATION,
    IC_ADMISSION,
    @DateTimeNow as Date_Last_Inserted 
FROM 
        VWSSTATIC.RIVM_IC_OPNAME
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.RIVM_IC_OPNAME)



END;
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'NICE_HOSPITAL'))
BEGIN

	CREATE TABLE [VWSSTATIC].[NICE_HOSPITAL](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS] [varchar](100) NULL,
		[MUNICIPALITY_CODE] [varchar](100) NULL,
		[MUNICIPALITY_NAME] [varchar](100) NULL,
		[SECURITY_REGION_CODE] [varchar](100) NULL,
		[SECURITY_REGION_NAME] [varchar](100) NULL,
		[HOSPITAL_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSINTER].[NICE_HOSPITAL] DROP CONSTRAINT [DATE_LAST_INSERTED]
GO

In [ ]:
 CREATE OR ALTER  PROCEDURE [dbo].[SP_NICE_HOSPITAL_INTER]
 AS
 BEGIN
 
 -- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

 -- MAIN SELECT AND INSERT INTO STATEMENT. FILTERED VALUES ON MAX DATELASTINSERTED.
 -- MOVE INFECTIOUS DATA FROM STAGING TO INTERMEDIATE TABLE.
 INSERT INTO [VWSINTER].[NICE_HOSPITAL](
 	[DATE_OF_REPORT]
     ,[DATE_OF_STATISTICS]
     ,[MUNICIPALITY_CODE]
     ,[MUNICIPALITY_NAME]
     ,[SECURITY_REGION_CODE]
     ,[SECURITY_REGION_NAME]
     ,[REPORTED]
     ,[HOSPITALIZED]
     ,DATE_LAST_INSERTED
 )
     SELECT
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT]
         ,CAST(A.[DATE_OF_STATISTICS] AS [DATETIME]) AS [DATE_OF_STATISTICS]
         ,ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
         ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
         ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
         ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [REPORTED]
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITALIZED]
         ,@DateTimeNow as Date_Last_Inserted
     FROM [VWSSTAGE].[NICE_HOSPITAL] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[NICE_HOSPITAL])
     GROUP BY ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
             ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
             ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
             ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
             ,A.[DATE_OF_REPORT]
             ,A.[DATE_OF_STATISTICS]

 INSERT INTO [VWSINTER].[NICE_HOSPITAL](
 	[DATE_OF_REPORT]
     ,[DATE_OF_STATISTICS]
     ,[MUNICIPALITY_CODE]
     ,[MUNICIPALITY_NAME]
     ,[SECURITY_REGION_CODE]
     ,[SECURITY_REGION_NAME]
     ,[REPORTED]
     ,[HOSPITALIZED]
     ,DATE_LAST_INSERTED
 )
     SELECT
         CAST(A.[DATE_OF_REPORT] AS [DATETIME]) AS [DATE_OF_REPORT]
         ,CAST(A.[DATE_OF_STATISTICS] AS [DATETIME]) AS [DATE_OF_STATISTICS]
         ,ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
         ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
         ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
         ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [REPORTED]
         ,CAST(SUM(CAST([HOSPITAL_ADMISSION] AS [FLOAT]) * ISNULL(B.[SHARES], 1.0)) AS [INT]) AS [HOSPITALIZED]
         ,@DateTimeNow as Date_Last_Inserted
     FROM [VWSSTATIC].[NICE_HOSPITAL] A
     LEFT JOIN (
         SELECT 
             * 
         FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
         WHERE [ACTIVE]=1
             AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
     ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
     WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[NICE_HOSPITAL])
     GROUP BY ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
             ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
             ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
             ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
             ,A.[DATE_OF_REPORT]
             ,A.[DATE_OF_STATISTICS]

 END;
 
GO

In [ ]:
IF NOT (EXISTS (SELECT * 
                 FROM INFORMATION_SCHEMA.TABLES 
                 WHERE TABLE_SCHEMA = 'VWSSTATIC' 
                 AND  TABLE_NAME = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP'))
BEGIN

	CREATE TABLE [VWSSTATIC].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP](
		[ID] [int] IDENTITY(1,1) NOT NULL,
		[VERSION] [varchar](100) NULL,
		[DATE_OF_REPORT] [varchar](100) NULL,
		[DATE_OF_STATISTICS_WEEK_START] [varchar](100) NULL,
		[AGE_GROUP] [varchar](100) NULL,
		[HOSPITAL_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[HOSPITAL_ADMISSION] [varchar](100) NULL,
		[IC_ADMISSION_NOTIFICATION] [varchar](100) NULL,
		[IC_ADMISSION] [varchar](100) NULL,
		[DATE_LAST_INSERTED] [datetime] Default GetDate()
	) ON [PRIMARY]
END

In [ ]:
ALTER TABLE [VWSINTER].[RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP] DROP CONSTRAINT DATE_LAST_INSERTED
GO

In [ ]:
CREATE   PROCEDURE [dbo].[SP_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_INTER]
AS
BEGIN

-- Zet de Date_Last_Inserted in een variabele
DECLARE @DateTimeNow AS DateTime = GETDATE()

INSERT INTO 
        VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
            [VERSION]
            ,[DATE_OF_REPORT]
            ,[DATE_OF_STATISTICS_WEEK_START]
            ,[AGE_GROUP]
            ,[HOSPITAL_ADMISSION_NOTIFICATION]
            ,[HOSPITAL_ADMISSION]
            ,[IC_ADMISSION_NOTIFICATION]
            ,[IC_ADMISSION]
            ,[DATE_LAST_INSERTED]
    )
   SELECT
         [VERSION]
        ,CAST([DATE_OF_REPORT] AS DATETIME)
        ,CAST([DATE_OF_STATISTICS_WEEK_START] AS DATE)
        ,[AGE_GROUP]
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION] AS INT)
		END
       ,@DateTimeNow as Date_Last_Inserted
    FROM 
        VWSSTAGE.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE 
        DATE_LAST_INSERTED = (
            SELECT MAX(DATE_LAST_INSERTED) 
            FROM VWSSTAGE.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)



INSERT INTO 
        VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
            [VERSION]
            ,[DATE_OF_REPORT]
            ,[DATE_OF_STATISTICS_WEEK_START]
            ,[AGE_GROUP]
            ,[HOSPITAL_ADMISSION_NOTIFICATION]
            ,[HOSPITAL_ADMISSION]
            ,[IC_ADMISSION_NOTIFICATION]
            ,[IC_ADMISSION]
            ,[DATE_LAST_INSERTED]
    )
   SELECT
         [VERSION]
        ,CAST([DATE_OF_REPORT] AS DATETIME)
        ,CAST([DATE_OF_STATISTICS_WEEK_START] AS DATE)
        ,[AGE_GROUP]
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([HOSPITAL_ADMISSION]) = 0 THEN NULL
			ELSE CAST([HOSPITAL_ADMISSION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION_NOTIFICATION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION_NOTIFICATION] AS INT)
		END
        ,CASE 
			WHEN LEN([IC_ADMISSION]) = 0 THEN NULL
			ELSE CAST([IC_ADMISSION] AS INT)
		END
       ,@DateTimeNow as Date_Last_Inserted
    FROM 
        VWSSTATIC.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE 
        DATE_LAST_INSERTED = (
            SELECT MAX(DATE_LAST_INSERTED) 
            FROM VWSSTATIC.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)


END;
GO

Nu de instructies toevoegen voor Datatino. Die bestaan uit het toevoegen van een source, workflow, process en twee dataflows. In de database zitten procedures om dit te doen: Upsert\_Source, Upsert\_Workflow en Upsert\_Process. Om dit compact te kunnen doen wordt hier eerst een nieuwe Stored Procedure aangemaakt die dit doet voor één nieuwe statische bron (dbo.Add\_Static\_File). Daarna wordt deze procedure 9 keer aangeroepen met de correcte parameters en uiteindelijk wordt de stored procedure weer uit de database verwijderd.

In [ ]:

DROP PROCEDURE IF EXISTS dbo.Add_Static_File;
GO

CREATE PROCEDURE dbo.Add_Static_File (@Source_content nvarchar(200)
                                        , @Workflow_name nvarchar(200)
                                        , @Target_name nvarchar(200)
                                        , @source_type nvarchar(200)
                                        , @ColumnMappingSource nvarchar(200)
                                        , @ColumnMappingTarget nvarchar(200))
AS
BEGIN

    -- Afgeleide variabelen maken
    DECLARE @Description nvarchar(200) = 'Load ' + @Workflow_Name 
    DECLARE @Source_Name nvarchar(200) = 'SOURCE_' + @Workflow_Name
    DECLARE @Process_Name nvarchar(200) = 'PROCESS_' + @Workflow_Name

    -- Aanamken DataFlow en WorkFlow, Source en Process
    Execute DATATINO_ORCHESTRATOR_1.UPSERT_WORKFLOW NULL,  @Workflow_Name ,  @Description, '0 12 * * MON',  0
    Execute DATATINO_ORCHESTRATOR_1.UPSERT_SOURCE NULL, @Source_Name, @description, @Source_content, @ColumnMappingSource, @ColumnMappingTarget, @target_name, @source_type, 'Web', 'SemiColon', 'RIVM'
    Execute DATATINO_ORCHESTRATOR_1.UPSERT_PROCESS NULL, @process_name, '', @source_name, '* * * * *', @workflow_name, 1

END

GO


Begin TRANSACTION

    DECLARE @Source_content nvarchar(200) = 'data.rivm.nl/data/vws/covid-19/covid-19_aantallen_gemeente_per_dag.csv' 
    DECLARE @Workflow_Name nvarchar(200) = 'RIVM_COVID_19_NUMBER_MUNICIPALITY_STATIC_FILE'
    DECLARE @Target_Name nvarchar(200) = 'VWSSTATIC.RIVM_COVID_19_NUMBER_MUNICIPALITY'
    DECLARE @SourceType nvarchar(200) = 'CsvFile'
    DECLARE @ColumnsMappingSource nvarchar(200) = 'Date_of_report|Date_of_publication|Municipality_code|Municipality_name|Province|Security_region_code|Security_region_name|Municipal_health_service|ROAZ_region|Total_reported|Deceased'
    DECLARE @ColumnMappingTarget nvarchar(200) = 'DATE_OF_REPORT|DATE_OF_PUBLICATION|MUNICIPALITY_CODE|MUNICIPALITY_NAME|PROVINCE|SECURITY_REGION_CODE|SECURITY_REGION_NAME|MUNICIPAL_HEALTH_SERVICE|ROAZ_REGION|TOTAL_REPORTED|DECEASED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_gehandicaptenzorg.csv' 
    SET @Workflow_Name = 'DISABILITY_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.DISABILITY'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Security_region_name|Total_cases_reported|Total_deceased_reported|Total_new_infected_locations_reported|Total_infected_locations_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|SECURITY_REGION_NAME|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|TOTAL_NEW_INFECTED_LOCATIONS_REPORTED|TOTAL_INFECTED_LOCATIONS_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_thuiswonend_70plus.csv' 
    SET @Workflow_Name = 'ELDERLY_AT_HOME_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.ELDERLY_AT_HOME'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Security_region|Total_cases_reported|Total_deceased_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|SECURITY_REGION|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_uitgevoerde_testen.csv' 
    SET @Workflow_Name = 'GGD_TESTED_PEOPLE_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.GGD_TESTED_PEOPLE'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|Security_region_code|Security_region_name|Tested_with_result|Tested_positive'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|SECURITY_REGION_CODE|SECURITY_REGION_NAME|TESTED_WITH_RESULT|TESTED_POSITIVE|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_verpleeghuizen.csv' 
    SET @Workflow_Name = 'RIVM_NURSING_HOMES_COMBINED_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.RIVM_NURSING_HOMES_COMBINED'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Date_of_report|Date_of_statistic_reported|Security_region_code|Total_cases_reported|Total_deceased_reported|Total_new_infected_locations_reported|Total_infected_locations_reported'
    SET @ColumnMappingTarget = 'DATE_OF_REPORT|DATE_OF_STATISTIC_REPORTED|SECURITY_REGION_CODE|TOTAL_CASES_REPORTED|TOTAL_DECEASED_REPORTED|TOTAL_NEW_INFECTED_LOCATIONS_REPORTED|TOTAL_INFECTED_LOCATIONS_REPORTED|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_reproductiegetal.json' 
    SET @Workflow_Name = 'RIVM_REPRODUCTION_NUMBER_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.RIVM_REPRODUCTION_NUMBER'
    SET @SourceType = 'JsonFile'
    SET @ColumnMappingSource = 'Date|Rt_low|Rt_avg|Rt_up|population'
    SET @ColumnMappingTarget = 'DATE|RT_LOW|RT_AVG|RT_UP|POPULATION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_ic_opnames.csv' 
    SET @Workflow_Name = 'RIVM_IC_OPNAME_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.RIVM_IC_OPNAME'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|IC_admission_notification|IC_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|IC_ADMISSION_NOTIFICATION|IC_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_ziekenhuisopnames.csv' 
    SET @Workflow_Name = 'NICE_HOSPITAL_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.NICE_HOSPITAL'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics|Municipality_code|Municipality_name|Security_region_code|Security_region_name|Hospital_admission_notification|Hospital_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS|MUNICIPALITY_CODE|MUNICIPALITY_NAME|SECURITY_REGION_CODE|SECURITY_REGION_NAME|HOSPITAL_ADMISSION_NOTIFICATION|HOSPITAL_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType  


    SET @Source_content = 'data.rivm.nl/data/vws/covid-19/COVID-19_ziekenhuis_ic_opnames_per_leeftijdsgroep.csv' 
    SET @Workflow_Name = 'RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_STATIC_FILE'
    SET @Target_Name = 'VWSSTATIC.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP'
    SET @SourceType = 'CsvFile'
    SET @ColumnMappingSource = 'Version|Date_of_report|Date_of_statistics_week_start|Age_group|Hospital_admission_notification|Hospital_admission|IC_admission_notification|IC_admission'
    SET @ColumnMappingTarget = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS_WEEK_START|AGE_GROUP|HOSPITAL_ADMISSION_NOTIFICATION|HOSPITAL_ADMISSION|IC_ADMISSION_NOTIFICATION|IC_ADMISSION|DATE_LAST_INSERTED=GETDATE'

    Execute dbo.Add_Static_File @Source_content, @Workflow_Name, @Target_Name, @SourceType, @ColumnMappingSource, @ColumnMappingTarget  

COMMIT


DROP PROCEDURE IF EXISTS dbo.Add_Static_File;
GO
